In [32]:
import numpy as np
# from numpy import genfromtxt
import matplotlib.pyplot as plt
import random
random.seed(0)  # please do not remove this line
import pandas as pd

### Question 2

In [33]:
def load_data_from_file(fileName="./data/advertising.csv"):
  data = pd.read_csv(fileName).to_numpy()
  features_X = data[:, :3]
  sales_Y = data[:, 3]
  
  N = features_X.shape[0]

  # Thêm một cột toàn giá trị 1 vào phía trước của X
  X_b = np.c_[np.ones((N, 1)), features_X]

  return X_b, sales_Y


features_X, _ = load_data_from_file()
print (features_X [:5 , :])

[[  1.  230.1  37.8  69.2]
 [  1.   44.5  39.3  45.1]
 [  1.   17.2  45.9  69.3]
 [  1.  151.5  41.3  58.5]
 [  1.  180.8  10.8  58.4]]


### Question 3

In [34]:
_, sales_Y = load_data_from_file()
print (sales_Y . shape )

(200,)


In [35]:
def generate_random_value(bound):
    return random.randint(-(bound // 2), bound // 2)

In [36]:
def create_individual (n = 4, bound =10):
    individual = [generate_random_value(bound) for i in range(n)]
    return individual
individual = create_individual()
print (individual)

[1, 1, -5, -1]


In [37]:
def predict(x, individual):
    theta = np.array(individual)
    y_hat = x.dot(theta)
    return y_hat

def compute_loss(y_hat, y):
    loss = np.multiply((y_hat - y), (y_hat - y)).mean()
    return loss

# individual 
def compute_fitness(individual, features_X, sales_Y):
    y_hat = predict(features_X, individual)
    loss = compute_loss(y_hat, sales_Y)
    fitness_value = 1 / (loss + 1)

    return fitness_value

### Question 4

In [38]:
features_X, _ = load_data_from_file()
features_X, sales_Y = load_data_from_file()
individual = [4.09, 4.82, 3.10, 4.02]
fitness_score = compute_fitness(individual, features_X, sales_Y)
print(fitness_score)

1.0185991537088997e-06


### Question 5

In [39]:
def crossover(individual1, individual2, crossover_rate=0.9):
    individual1_new = individual1.copy()
    individual2_new = individual2.copy()

    for i in range(len(individual1_new)):
        if random.random() < crossover_rate:
            individual1_new[i], individual2_new[i] = individual2_new[i], individual1_new[i]

    return individual1_new, individual2_new

In [40]:
# question 5
individual1 = [4.09, 4.82, 3.10, 4.02]
individual2 = [3.44, 2.57, -0.79, -2.41]

individual1, individual2 = crossover(individual1, individual2, 2.0)
print("individual1: ", individual1)
print("individual2: ", individual2)

individual1:  [3.44, 2.57, -0.79, -2.41]
individual2:  [4.09, 4.82, 3.1, 4.02]


### Question 6:

In [41]:
def mutate(individual, mutation_rate=0.05):
    individual_m = individual.copy()

    return [1 - val if random.random() < mutation_rate else val for val in individual_m]

In [42]:
# Question 6
before_individual = [4.09, 4.82, 3.10, 4.02]
after_individual = mutate(individual, mutation_rate=2.0)
print("After mutation: ", after_individual)
print(before_individual == after_individual)

After mutation:  [-3.09, -3.8200000000000003, -2.1, -3.0199999999999996]
False


In [43]:
def initializePopulation(m) :
    population = [create_individual() for _ in range(m)]
    return population

#### Thiết kế hàm selection : không được chọn 2 phần tử trùng nhau , phần tử thứ 2 > 1 

In [44]:
# population được sorted tăng dần
def select(population_sorted, m = 100):
    index1 = random.randint(0, m - 1)
    while True:
        index2 = random.randint(0, m - 1)
        if index1 != index2:
            break

    if index2 > index1: 
        return population_sorted[index2]
    else:
        return population_sorted[index1]

In [45]:
# population được sắp xếp tăng dần
def create_new_population(population,features_X, sales_Y, elitism = 2, gen = 1):
    m = len(population)
    sorted_population = sorted(population, key=lambda individual: compute_fitness(individual, features_X, sales_Y))

    y_hat = predict(features_X, sorted_population[m - 1])
    if gen %1 == 0:
        print("Best loss :", compute_loss(y_hat, sales_Y),
              "with chromsome: ", sorted_population[m - 1])
        
    new_population = []
    new_population.extend(population[-elitism:])
    while len(new_population) < m:
        # selection
        parent1 = select(population_sorted=sorted_population, m=100)
        parent2 = select(population_sorted=sorted_population, m=100)
        
        # crossover
        child1, child2 = crossover(parent1, parent2)
        
        # mutation
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))

    return new_population[: m]

### Question 7

In [46]:
# Question 7
individual1 = [4.09, 4.82, 3.10, 4.02]
individual2 = [3.44, 2.57, -0.79, -2.41]
old_population = [individual1, individual2]
new_population, _ = create_new_population(old_population,features_X, sales_Y, elitism=2, gen=1)

Best loss : 123415.05152880498 with chromsome:  [3.44, 2.57, -0.79, -2.41]
